In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd
# from grouping_using_cos_sim import grouping_

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:

# Load your data
path = "/content/drive/MyDrive/newMind_task/merged_dataset.csv"
grouped_df = pd.read_csv(path)
grouped_df = grouped_df
# Define your label mapping



label_mapping = {'Claim': 0, 'Counterclaim': 1, 'Evidence': 2, 'Rebuttal': 3}
grouped_df['type'] = grouped_df['type'].map(label_mapping)

# Split the dataset into train, val, and test sets
train_df = grouped_df.sample(frac=0.8, random_state=42)

remaining_df = grouped_df.drop(train_df.index)
val_df = remaining_df.sample(frac=0.5, random_state=42)
test_df = remaining_df.drop(val_df.index)

# Convert the DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)




In [ ]:

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=4)


# Define a function to concatenate and tokenize the text
def preprocess_function(examples):
    combined_texts = [opinion + " " + topic for opinion, topic in zip(examples['opinions'], examples['topics'])]
    # to be sure that the inputs are same length we need to trim the long strings and padding the short ones with tokens
    tokenized_inputs = tokenizer(combined_texts, padding="max_length", truncation=True)
    tokenized_inputs['labels'] = examples['type']
    return tokenized_inputs

# Apply tokenization to the datasets
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_val_dataset = val_dataset.map(preprocess_function, batched=True)
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)



# Define a function to compute metrics
def compute_metrics(p):
    # Get the predicted labels by finding the index of the highest logit
    preds = np.argmax(p.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='weighted')
    acc = accuracy_score(p.label_ids, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}


training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/newMind_task/results',
    eval_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    compute_metrics=compute_metrics
)

# Start training again
trainer.train()


# Save the best model and tokenizer
trainer.save_model('/content/drive/MyDrive/newMind_task/best_model')
tokenizer.save_pretrained('/content/drive/MyDrive/newMind_task/best_model')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Map:   0%|          | 0/21502 [00:00<?, ? examples/s]

Map:   0%|          | 0/2688 [00:00<?, ? examples/s]

Map:   0%|          | 0/2687 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.551600,0.511101,0.820685,0.820832,0.821403,0.820685
2,0.435200,0.476767,0.829613,0.826201,0.834372,0.829613
3,0.349300,0.502938,0.837426,0.836927,0.837680,0.837426


TrainOutput(global_step=4032, training_loss=0.47168047205796315, metrics={'train_runtime': 1562.5084, 'train_samples_per_second': 41.284, 'train_steps_per_second': 2.58, 'total_flos': 1.6972546510872576e+16, 'train_loss': 0.47168047205796315, 'epoch': 3.0})

In [ ]:
# Load the best saved model
best_model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/newMind_task/best_model')
best_tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/newMind_task/best_model')
# Example input text
text = "i hate u so much"
# Tokenize the input text
inputs = best_tokenizer(text, padding=True, truncation=True, return_tensors="pt")

# Make predictions
outputs = best_model(**inputs)
logits = outputs.logits
predictions = logits.argmax(dim=-1)

# Define your label mapping
label_mapping = {0: 'Claim', 1: 'Counterclaim', 2: 'Evidence', 3: 'Rebuttal'}

# Map the prediction to the label
predicted_label = label_mapping[predictions.item()]
print(predicted_label)

Claim
